In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from hyperopt import hp, tpe, Trials, fmin, space_eval
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random
import feather
import riiideducation

In [2]:
data_types_dict = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float16',
    'prior_question_had_explanation': 'boolean'
}

In [3]:
DIR = "../input/riiid-test-answer-prediction/"
#train = pd.read_feather("train.feather")
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                       nrows=10**3,
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict, 
                       index_col = 0)
lectures = pd.read_csv(DIR+"lectures.csv")
questions = pd.read_csv(DIR+"questions.csv")
example_test = pd.read_csv(DIR+"example_test.csv")

In [4]:
train[train.user_id==115]["prior_question_had_explanation"].value_counts(sort=True)[1]

6

In [5]:
train[train.user_id==115]

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,36992.0,False
2,118363,115,128,0,0,0,1,55008.0,False
3,131167,115,7860,0,3,0,1,19008.0,False
4,137965,115,7922,0,4,1,1,11000.0,False
5,157063,115,156,0,5,2,1,5000.0,False
6,176092,115,51,0,6,0,1,16992.0,False
7,194190,115,50,0,7,3,1,16992.0,False
8,212463,115,7896,0,8,2,1,16000.0,False


# Feature engineering
- reference: https://www.kaggle.com/taichin/final-my-model/notebook

In [6]:
def feature_engineering(user_sample, test_set=False):
    #content_id_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    #exp_false_count = user_sample["prior_question_had_explanation"].value_counts(sort=True)[0]
    #exp_true_count = user_sample["prior_question_had_explanation"].value_counts(sort=True)[1]
    
    all_df = []
    features = {"user_id": 0}
    features["user_id"] = user_sample.iloc[0]["user_id"]
    features["mean_accuracy"] = user_sample["answered_correctly"].sum() / user_sample.shape[0]
    features["timestamp_diff_ave"] = np.mean(user_sample["timestamp"].diff(1))
    features["timestamp_diff_std"] = np.std(user_sample["timestamp"].diff(1))
    #features["prior_explanation_true_ratio"] = exp_true_count / (exp_true_count + exp_false_count)
    #features["prior_question_elapsed_time_ave"] = np.mean(user_sample["prior_question_elapsed_time"].dropna())
    #features["prior_question_elapsed_time_std"] = np.std(user_sample["prior_question_elapsed_time"].dropna())
    all_df.append(features)
    return all_df

In [7]:
def get_data(df):
    compiled_df = []

    for i, (ins_id, user_sample) in tqdm(enumerate(df.groupby('user_id', sort=False)), total=df.user_id.nunique(), desc='user_id', position=0):
        compiled_df += feature_engineering(user_sample)
    #del train
    #for ins_id, user_sample in tqdm(test.groupby('user_id', sort=False), total=test.user_id.nunique(), desc='user_id', position=0):
    #    test_data, val_data = get_data(user_sample, test_set=True)
    #    compiled_test.append(test_data)
    #    compiled_val += val_data
    #del test
    reduce_df = pd.DataFrame(compiled_df)
    #reduce_test = pd.DataFrame(compiled_test)
    #reduce_val = pd.DataFrame(compiled_val)

    #categoricals = ['session_title']
    return reduce_df
new_train = get_data(train)
new_train

,user_id,mean_accuracy,timestamp_diff_ave,timestamp_diff_std
0,115,0.695652,1.484645e+07,9.822442e+07
1,124,0.233333,1.970079e+04,2.626478e+04
2,2746,0.500000,4.397142e+04,4.627572e+04
3,5382,0.632812,1.654765e+07,8.658986e+07
4,8623,0.598214,7.768817e+06,3.560077e+07
5,8701,0.588235,9.820569e+04,2.239976e+05
6,12741,0.538745,1.653884e+07,1.925863e+08
7,13134,0.662234,4.024665e+07,4.066932e+08


# modelling

In [8]:
def modelling_lgb(new_train, target, new_test, target, layer):
    
    X_train = new_train.copy()
    y_train = target_train[target].copy()
    X_test = new_test.copy()
    
    params = {'objective': 'binary', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, 
               "num_leaves": 10, 'random_seed':44, 'max_depth': 5} 
        
    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    
    pred_value = np.zeros(X_test.shape[0])
    valid = np.zeros([X_train.shape[0]])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_valid2 = X_train.iloc[test_index,:]
        y_valid2 = y_train.iloc[test_index]
        
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
        clf = lgb.train(params, lgb_train,valid_sets=[lgb_train, lgb_eval], 
               num_boost_round=10000,early_stopping_rounds=25,verbose_eval = 0) 

        valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
        valid[test_index] = valid_predict
        pred_value += clf.predict(X_test, num_iteration = clf.best_iteration) / n_folds
            
    score = roc_auc_score(y_train, valid)
            
    return valid, pred_value, score

SyntaxError: duplicate argument 'target' in function definition (<ipython-input-8-420e25a665a9>, line 4)

# prediction